In [8]:
import pickle
import time

from selenium import webdriver
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
import json

keywords = ["산문", "에세이", "수필", "감성에세이", "일상에세이", "생각", "인생"]

for keyword in keywords:
    print(keyword)

    tq = tqdm(desc=f"{keyword} 가져오는 중...", unit="pages")

    webdriver_options = webdriver.ChromeOptions()
    webdriver_options.add_argument('headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver_options)

    # driver.maximize_window()
    driver.implicitly_wait(2)

    ## 산문
    url = "https://brunch.co.kr/keyword/" + keyword
    driver.get(url)

    start_page = 1
    save_count = 2
    result = {
        "source": "브런치",
        "url": url,
        "desc": f"브런치 {keyword} 키워드 크롤링 데이터",
        "note": "지금은 열시반",
        "data": []
    }
    data = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    SCROLL_PAUSE_TIME = 2

    skipped = []
    while True:
        for page in range(start_page, start_page + 19):
            try:
                # 감성 에세이 페이지 내에 있는 각 문서별로 접근
                acting_point = driver.find_element_by_css_selector(
                        f'#wrapArticle > div.wrap_article_list.\#keyword_related_contents > ul > li:nth-child({page}) > a'
                )
                driver.execute_script("arguments[0].click();", acting_point)

                driver.switch_to.window(driver.window_handles[-1])
                driver.get_window_position(driver.window_handles[-1])
                driver.implicitly_wait(1)
            except:
                pass

            tq.update()
            time.sleep(1)

            # 각 문서 제목 저장, 존재하지 않을 시 누락 값 전달
            try:
                author = driver.current_url.split("/")[-2]
                assert author != ""
                title = driver.find_element_by_class_name("cover_title").text

                text_parent = driver.find_element_by_class_name("wrap_body")
                text_children = text_parent.find_elements_by_tag_name("div")
                for child in reversed(text_children):
                    driver.execute_script("""var element = arguments[0];element.parentNode.removeChild(element)""",
                                          child)

                text = text_parent.text

            except:
                skipped.append(driver.current_url)
                tq.set_postfix({"skipped": len(skipped)})

            else:
                data.append({
                    "class": keyword,
                    "title": title,
                    "author": author,
                    "text": text,
                    "url": driver.current_url
                })

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.get_window_position(driver.window_handles[0])

        # 무한 스크롤
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("스크롤 끝! 고생했어~")
                print(f"{len(skipped)}개 스킵함 ㅠㅠㅠ")
                json.dump(skipped, open("./skipped.json", 'w', encoding='utf-8'), ensure_ascii=False, indent=2)
                break
        except:
            print(f"{keyword} 하다가 아무튼 끝난듯... 그만함")
            break

        last_height = new_height
        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤 내린 후 페이지 목록 전달
        start_page += 19

        # 19개 이상 탐색했을때 저장
        if start_page // 19 == save_count:
            save_count += 1
            # 결과 리스트 저장해둘 경로 설정

        result["data"] = data
        json.dump(result, open(f"./brunch/{keyword}.json", 'w', encoding='utf-8'), ensure_ascii=False, indent=2)
        pickle.dump(result, open(f"./brunch/{keyword}.pickle", "wb"))

    print(f"{keyword}에서 {len(data)}개 뽑음!!!")